In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# Load the datasets
merged_df = pd.read_csv("datasets/train_merged_df.csv")

## Feature Engineering: Grouping the data by bidder_id

In [3]:
# Get the number of bids per bidder
bids_per_bidder = merged_df.groupby('bidder_id').size().reset_index(name='num_bids')

In [4]:
# Get the number of unique countries per bidder
unique_countries_per_bidder = merged_df.groupby('bidder_id')['country'].nunique().reset_index(name='num_countries')

In [5]:
# Get the number of unique merchandise types per bidder
unique_merchandise_per_bidder = merged_df.groupby('bidder_id')['merchandise'].nunique().reset_index(name='num_merchandise')

In [6]:
# Get the number of unique URLs per bidder
unique_urls_per_bidder = merged_df.groupby('bidder_id')['url'].nunique().reset_index(name='num_urls')

In [7]:
# Get the number of unique IPs per bidder
unique_ips_per_bidder = merged_df.groupby('bidder_id')['ip'].nunique().reset_index(name='num_ips')

In [8]:
# Get the number of unique devices per bidder
unique_devices_per_bidder = merged_df.groupby('bidder_id')['device'].nunique().reset_index(name='num_devices')

In [9]:
# Get the number of auctions joined per bidder
auctions_joined_per_bidder = merged_df.groupby('bidder_id')['auction'].nunique().reset_index(name='num_auctions')

In [10]:
# Get the total duration between the first and last bid for each bidder
total_duration = merged_df.groupby('bidder_id')['time'].agg(lambda x: x.max() - x.min()).reset_index(name='total_duration')
total_duration['total_duration'] = total_duration['total_duration'] / 1e9  # Convert to smaller values

In [11]:
# Get the majority outcome per bidder
majority_outcome_per_bidder = merged_df.groupby('bidder_id')['outcome'].agg(lambda x: x.mode()[0]).reset_index(name='outcome')

In [12]:
# Merge all the features into a single DataFrame
featured_df = bids_per_bidder
featured_df = pd.merge(featured_df, unique_countries_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, unique_merchandise_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, unique_urls_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, unique_ips_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, unique_devices_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, auctions_joined_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, total_duration, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, majority_outcome_per_bidder, on='bidder_id', how='left')

In [13]:
# Visualizing the table by printing the data for 10 bidders 
print(featured_df.head(10))

                               bidder_id  num_bids  num_countries  \
0  001068c415025a009fee375a12cff4fcnht8y         1              1   
1  0030a2dd87ad2733e0873062e4f83954mkj86         1              1   
2  00a0517965f18610417ee784a05f494d4dw6e       141             16   
3  00cc97158e6f4cb8eac3c0075918b7ffi5k8o         3              2   
4  01067975436d123f717ee5aba0dd4bbfa0937       543             71   
5  012441119bcf83b23d4768bb72cea6d6carua        23             10   
6  01255c2c7c5578c186873422fc00fd7afwk8k        65              5   
7  01349288df20199905e719f0ff7ee771nwryj        70              4   
8  0141844fc725a85e38e12cc02fcdbbe8ofqxv        18              6   
9  0181b91f58e6cbaa832235e2879c9c00ovrzi        15              3   

   num_merchandise  num_urls  num_ips  num_devices  num_auctions  \
0                1         1        1            1             1   
1                1         1        1            1             1   
2                1        84      10

In [14]:
# Normalizing the features
features_to_scale = ['num_bids', 'num_countries', 'num_merchandise', 'num_urls', 'num_ips', 'num_devices', 'num_auctions', 'total_duration']
scaler = StandardScaler()
featured_df[features_to_scale] = scaler.fit_transform(featured_df[features_to_scale])

In [15]:
featured_df.head()

,bidder_id,num_bids,num_countries,num_merchandise,num_urls,num_ips,num_devices,num_auctions,total_duration,outcome
0,001068c415025a009fee375a12cff4fcnht8y,-0.105254,-0.525142,-0.022456,-0.128930,-0.140742,-0.413474,-0.418184,-0.971009,0.0
1,0030a2dd87ad2733e0873062e4f83954mkj86,-0.105254,-0.525142,-0.022456,-0.128930,-0.140742,-0.413474,-0.418184,-0.971009,0.0
2,00a0517965f18610417ee784a05f494d4dw6e,-0.095728,0.115044,-0.022456,-0.097918,-0.119444,-0.061818,0.077506,1.398677,0.0
3,00cc97158e6f4cb8eac3c0075918b7ffi5k8o,-0.105117,-0.482463,-0.022456,-0.128930,-0.140324,-0.402817,-0.404603,0.724700,0.0
4,01067975436d123f717ee5aba0dd4bbfa0937,-0.068377,2.462396,-0.022456,-0.127809,-0.057847,0.460336,-0.309539,-0.551787,0.0


In [16]:
# Splitting the dataset into training and validation sets
X = featured_df.drop(['bidder_id','outcome'], axis=1)  # Features
y = featured_df['outcome']  # Target variable (1 - Robot, 0 - Human)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Check dataset balance or not
y_train.value_counts()

outcome
0.0    1511
1.0      76
Name: count, dtype: int64

In [18]:
# Handle Imbalance - Oversampling using SMOTE
from imblearn.over_sampling import SMOTE

smt = SMOTE()
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

In [19]:
# Check outcome 
print(X_train_sm.shape, y_train_sm.shape)
y_train_sm.value_counts()

(3022, 8) (3022,)


outcome
0.0    1511
1.0    1511
Name: count, dtype: int64

In [20]:
# Save into csv for FNN training
train_df = pd.concat([X_train_sm, y_train_sm], axis=1)
train_df.head()
train_df.to_csv("datasets/tabular_train_data.csv", index=False)

In [21]:
# Save the validation dataset also for FNN evaluation
val_df = pd.concat([X_test, y_test], axis=1)
val_df.head()
val_df.to_csv("datasets/tabular_validation_data.csv", index=False)

## Prepare preprocessed tabular test dataset

In [22]:
# Load the datasets
merged_df = pd.read_csv("datasets/test_merged_df.csv")

In [23]:
# Get the number of bids per bidder
bids_per_bidder = merged_df.groupby('bidder_id').size().reset_index(name='num_bids')

# Get the number of unique countries per bidder
unique_countries_per_bidder = merged_df.groupby('bidder_id')['country'].nunique().reset_index(name='num_countries')

# Get the number of unique merchandise types per bidder
unique_merchandise_per_bidder = merged_df.groupby('bidder_id')['merchandise'].nunique().reset_index(name='num_merchandise')

# Get the number of unique URLs per bidder
unique_urls_per_bidder = merged_df.groupby('bidder_id')['url'].nunique().reset_index(name='num_urls')

# Get the number of unique IPs per bidder
unique_ips_per_bidder = merged_df.groupby('bidder_id')['ip'].nunique().reset_index(name='num_ips')

# Get the number of unique devices per bidder
unique_devices_per_bidder = merged_df.groupby('bidder_id')['device'].nunique().reset_index(name='num_devices')

# Get the number of auctions joined per bidder
auctions_joined_per_bidder = merged_df.groupby('bidder_id')['auction'].nunique().reset_index(name='num_auctions')

# Get the total duration between the first and last bid for each bidder
total_duration = merged_df.groupby('bidder_id')['time'].agg(lambda x: x.max() - x.min()).reset_index(name='total_duration')
total_duration['total_duration'] = total_duration['total_duration'] / 1e9  # Convert to smaller values

In [24]:
# Merge all the features into a single DataFrame
featured_df = bids_per_bidder
featured_df = pd.merge(featured_df, unique_countries_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, unique_merchandise_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, unique_urls_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, unique_ips_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, unique_devices_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, auctions_joined_per_bidder, on='bidder_id', how='left')
featured_df = pd.merge(featured_df, total_duration, on='bidder_id', how='left')

In [25]:
# Visualizing the table by printing the data for 10 bidders 
print(featured_df.shape)
print(featured_df.head(10))

(4630, 9)
                               bidder_id  num_bids  num_countries  \
0  002d229ffb247009810828f648afc2ef593rb         2              1   
1  003180b29c6a5f8f1d84a6b7b6f7be57tjj1o         3              1   
2  00486a11dff552c4bd7696265724ff81yeo9v        20              1   
3  0051aef3fdeacdadba664b9b3b07e04e4coc6        68              4   
4  0053b78cde37c4384a20d2da9aa4272aym4pb     10939             73   
5  0061edfc5b07ff3d70d693883a38d370oy4fs       134              4   
6  00862324eb508ca5202b6d4e5f1a80fc3t3lp         5              1   
7  009479273c288b1dd096dc3087653499lrx3c         1              1   
8  009cee781e8414f7fb55b2f92157e9dbu0y6o         5              3   
9  00a79ebd15f0b24a0a3b5794457cd8ed7dng1       398             13   

   num_merchandise  num_urls  num_ips  num_devices  num_auctions  \
0                1         1        1            2             1   
1                1         2        3            3             3   
2                1        

In [26]:
# Normalizing the features
features_to_scale = ['num_bids', 'num_countries', 'num_merchandise', 'num_urls', 'num_ips', 'num_devices', 'num_auctions', 'total_duration']
featured_df[features_to_scale] = scaler.transform(featured_df[features_to_scale])

In [27]:
featured_df.head()

,bidder_id,num_bids,num_countries,num_merchandise,num_urls,num_ips,num_devices,num_auctions,total_duration
0,002d229ffb247009810828f648afc2ef593rb,-0.105185,-0.525142,-0.022456,-0.128930,-0.140742,-0.408146,-0.418184,-0.971005
1,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,-0.105117,-0.525142,-0.022456,-0.128557,-0.140324,-0.402817,-0.404603,1.063659
2,00486a11dff552c4bd7696265724ff81yeo9v,-0.103961,-0.525142,-0.022456,-0.126688,-0.138863,-0.376177,-0.336700,1.384308
3,0051aef3fdeacdadba664b9b3b07e04e4coc6,-0.100695,-0.397105,-0.022456,-0.128183,-0.138863,-0.386833,-0.391022,-0.633051
4,0053b78cde37c4384a20d2da9aa4272aym4pb,0.638939,2.547754,-0.022456,0.594453,0.266423,2.341159,2.345457,1.413016


In [28]:
# Save into csv for FNN training
featured_df.to_csv("datasets/tabular_test_data.csv", index=False)